### Autoencoder_gan

In [ ]:
import os
import sys

def add_to_path(new_path: str):
    module_path = os.path.abspath(os.path.join(new_path))
    if module_path not in sys.path:
        sys.path.append(module_path)

is_colab = False

if is_colab:
    !git clone https://github.com/pdkary/Karys.git
    !cd Karys && git fetch && git pull
    !cd Karys && pip install -r requirements.txt --quiet --user
    add_to_path('Karys/')
    from google.colab import drive
    drive.mount('/content/drive')
    !cd Karys && pip install -r requirements.txt --quiet --user
else:
    add_to_path('../../')
    !pip install -r ../../requirements.txt

In [ ]:
from data.configs.ImageDataConfig import ImageDataConfig
from data.wrappers.ImageDataWrapper import ImageDataWrapper
import numpy as np

if is_colab:
    base_path = "drive/MyDrive/Colab/Seefood/Fruit"
else:
    base_path = "../discriminator/test_input/Fruit"

image_config = ImageDataConfig(image_shape=(64,64,3),image_type=".jpg", preview_rows=4, preview_cols=2, load_n_percent=2)

only_some_fruit = ['Watermelon', 'Peach', 'Pomegranate', 'Pineapple', 'Orange', 'Banana', 'Strawberry', 'Apple Braeburn', 'Lemon', 'Raspberry', 'Avocado', 'Pepper Green', 'Cherry', 'Limes', 'Cantaloupe', 'Corn', 'Pear']
data_wrapper = ImageDataWrapper.load_from_labelled_directories(base_path + '/', image_config, validation_percentage=0.1, use_dirs=only_some_fruit)
classification_labels = list(set(data_wrapper.image_labels.values()))

##Quick stats about loaded images
print("----------LOADED IMAGE CATEGORIES----------")
print(classification_labels)
print("NUM IMAGES: ",len(data_wrapper.image_labels))

print("----------LOADED IMAGE STATS----------")
value_count_dict = {v:0 for v in set(data_wrapper.image_labels.values())}
for x in data_wrapper.image_labels.values():
    value_count_dict[x]+=1
print(value_count_dict)
print("Max: ",np.max(list(value_count_dict.values())), max(value_count_dict,key=value_count_dict.get))
print("Min: ",np.min(list(value_count_dict.values())), min(value_count_dict,key=value_count_dict.get))
print("Mean: ",np.mean(list(value_count_dict.values())))
print("Median: ",np.median(list(value_count_dict.values())))
print("Deviation: ",np.std(list(value_count_dict.values())))

#### We're going to load a previous model from the encoder example... run it yourself if you want one

In [ ]:
from data.saved_models.SavedModelService import SavedModelService
from models.EncoderModel import EncoderModel
from models.ClassificationModel import ClassificationModel
from models.EncodedClassificationModel import EncodedClassificationModel
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction

if is_colab:
  model_output_path = "drive/MyDrive/Colab/Seefood/models"
  model_reference_path = "drive/MyDrive/Colab/Seefood/models/model_ref.json"
else:
  model_output_path = "./test_model_output"
  model_reference_path = "../encoder/test_model_output/model_ref.json"

saved_model_service = SavedModelService(model_reference_path)

encoder_optimizer = Adam(learning_rate=5e-5, decay=1e-9)
classifier_optimizer = Adam(learning_rate=5e-5, decay=1e-9)
encoder_loss = CategoricalCrossentropy(reduction=Reduction.SUM)
classifier_loss = CategoricalCrossentropy(from_logits=True, reduction=Reduction.SUM)

encoder_model: EncoderModel = saved_model_service.load_model_by_name("FruitEncoder", EncoderModel, encoder_optimizer, encoder_loss)
classification_model: ClassificationModel = saved_model_service.load_model_by_name("EncodedFruitClassifier", ClassificationModel, classification_labels, optimizer=classifier_optimizer, loss=classifier_loss)

encoded_classifier: EncodedClassificationModel = EncodedClassificationModel(encoder_model, classification_model)
encoded_classifier.build()

In [ ]:
from models.GenerationModel import GenerationModel
from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, Activation, Flatten, Reshape, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction
from tensorflow_addons.layers import InstanceNormalization

ENCODED_DIM = encoded_classifier.encoder.output_shape[-1]

a = 0.08
gen_layers = [
    Dense(512), Activation('relu'),
    Dense(4096), Activation('relu'),
    Reshape((2,2,1024)),
    UpSampling2D(),
    Conv2D(1024,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(1024,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(1024,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    UpSampling2D(),
    Conv2D(256,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(256,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(256,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    UpSampling2D(),
    Conv2D(64,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(64,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(64,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    UpSampling2D(),
    Conv2D(32,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(32,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(32,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    UpSampling2D(),
    Conv2D(16,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(16,3,padding="same"), BatchNormalization(), LeakyReLU(a),
    Conv2D(3,3,padding="same"), Activation('sigmoid'),
]

optimizer = Adam(learning_rate=5e-4)
loss = CategoricalCrossentropy(from_logits=True, reduction=Reduction.SUM)
generator = GenerationModel(ENCODED_DIM, image_config.image_shape,gen_layers,optimizer,loss)
generator.build()

In [ ]:
from plotting.TrainPlotter import TrainPlotter
from trainers.AutoEncoderTrainer import AutoEncoderTrainer

epochs=11
trains_per_test=5

train_columns = ["Encoder Loss", "Classifier Loss", "Generator Loss", "Test Encoder Loss", "Test Classifier Loss", "Test Generator Loss"]
loss_plot = TrainPlotter(moving_average_size=100, labels=train_columns)
trainer = AutoEncoderTrainer(encoded_classifier, generator, data_wrapper)

if is_colab:
  image_output_path = "drive/MyDrive/Colab/Seefood/output"
else:
  image_output_path = "./test_output"

e_test_loss, c_test_loss, g_test_loss = 0, 0, 0
for i in range(epochs):
  loss_plot.start_epoch()  
  e_loss, c_loss, g_loss = trainer.train(5, 1)

  if i % trains_per_test == 0 and i != 0:
    e_test_loss, c_test_loss, g_test_loss = trainer.test(16,1)
    encoding_real_filename = image_output_path + "/encoded-real-" + str(i) + ".jpg"
    encoding_gen_filename = image_output_path + "/encoded-gen-" + str(i) + ".jpg"
    classification_real_filename = image_output_path + "/classified-real-" + str(i) + ".jpg"
    classification_gen_filename = image_output_path + "/classified-gen-" + str(i) + ".jpg"


    data_wrapper.save_generated_images(encoding_real_filename, trainer.most_recent_generated)
    data_wrapper.save_encoded_images(encoding_real_filename, trainer.most_recent_real_encoding, img_size=32)
    data_wrapper.save_encoded_images(encoding_gen_filename, trainer.most_recent_gen_encoding, img_size=32)
    data_wrapper.save_classified_images(classification_real_filename, trainer.most_recent_real_classification, img_size=32)
    data_wrapper.save_classified_images(classification_gen_filename, trainer.most_recent_gen_classification, img_size=32)
      
  loss_plot.batch_update([e_loss, c_loss, g_loss, e_test_loss, c_test_loss, g_test_loss])
  loss_plot.log_epoch()